# 05: Advanced Visualization

**Level:** Intermediate | **Time:** ~15 min

## Overview
Create publication-quality figures with custom styling.

In [ ]:
import sys
sys.path.insert(0, '..')
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path
from src.config import CONFIG
from src.hierarchical import hierarchical_case2
from src.viz_paper import plot_corner, plot_timeseries
import matplotlib as mpl

np.random.seed(42)
output_dir = Path('figures')
output_dir.mkdir(exist_ok=True)

# Set publication style
mpl.rcParams['font.family'] = 'serif'
mpl.rcParams['font.size'] = 11
mpl.rcParams['axes.linewidth'] = 1.5
mpl.rcParams['grid.alpha'] = 0.3

print('✓ Setup complete')

## 1. Run Calibration

In [ ]:
results = hierarchical_case2(CONFIG)
samples = results.tmcmc_M1.samples[-1]
print(f'✓ Calibration complete: {len(samples)} samples')

## 2. Corner Plot (Parameter Correlations)

In [ ]:
from corner import corner  # pip install corner

labels = ['a₁₁', 'a₁₂', 'a₂₂', 'b₁', 'b₂']
truths = CONFIG.get('theta_true', [None]*5)[0:5]

fig = corner(samples, labels=labels, truths=truths,
             show_titles=True, title_fmt='.3f',
             quantiles=[0.16, 0.5, 0.84],
             levels=(0.68, 0.95),
             color='steelblue',
             truth_color='red',
             hist_kwargs={'density': True})

plt.savefig(output_dir / '05_corner.png', dpi=300, bbox_inches='tight')
plt.show()
print('✓ Corner plot saved (300 DPI)')

## 3. Time Series with Uncertainty Bands

In [ ]:
from src.posterior_simulator_tsm import generate_predictions

n_samples = 100
predictions = generate_predictions(samples[:n_samples], CONFIG['M1'])
mean = np.mean(predictions, axis=0)
lower = np.percentile(predictions, 2.5, axis=0)
upper = np.percentile(predictions, 97.5, axis=0)

t = np.linspace(0, CONFIG['M1']['maxtimestep'] * CONFIG['M1']['dt'], len(mean))

fig, axes = plt.subplots(2, 2, figsize=(12, 10))
species = ['Species 1', 'Species 2', 'Species 3', 'Species 4']

for idx, (ax, name) in enumerate(zip(axes.flat, species)):
    if idx < mean.shape[1]:
        ax.fill_between(t, lower[:, idx], upper[:, idx], 
                        alpha=0.3, color='steelblue', label='95% CI')
        ax.plot(t, mean[:, idx], 'b-', linewidth=2, label='Mean')
        ax.set_xlabel('Time', fontsize=11)
        ax.set_ylabel('Volume Fraction', fontsize=11)
        ax.set_title(name, fontsize=12, fontweight='bold')
        ax.legend(frameon=True, fancybox=True)
        ax.grid(True, linestyle='--', alpha=0.3)

plt.tight_layout()
plt.savefig(output_dir / '05_timeseries.png', dpi=300, bbox_inches='tight')
plt.show()
print('✓ Time series plot saved')

## 4. Custom Matplotlib Styling

In [ ]:
# Professional color schemes
colors = {
    'primary': '#2E86AB',
    'secondary': '#A23B72',
    'accent': '#F18F01',
    'success': '#06A77D',
    'warning': '#D00000'
}

# Example: Styled posterior plot
fig, axes = plt.subplots(1, 5, figsize=(15, 3))
for i, ax in enumerate(axes):
    ax.hist(samples[:, i], bins=30, density=True, 
            color=colors['primary'], alpha=0.7, edgecolor='black', linewidth=1.2)
    ax.axvline(truths[i] if truths[i] else 0, 
               color=colors['warning'], linestyle='--', linewidth=2.5, label='True')
    ax.set_xlabel(labels[i], fontsize=12, fontweight='bold')
    ax.set_ylabel('Density' if i == 0 else '', fontsize=11)
    ax.spines['top'].set_visible(False)
    ax.spines['right'].set_visible(False)
    if i == 0:
        ax.legend(frameon=False, fontsize=10)
    ax.grid(axis='y', alpha=0.3, linestyle=':')

plt.suptitle('Posterior Distributions', fontsize=14, fontweight='bold', y=1.02)
plt.tight_layout()
plt.savefig(output_dir / '05_styled_posterior.png', dpi=300, bbox_inches='tight')
plt.show()
print('✓ Styled posterior plot saved')

## 5. Export for Publications

In [ ]:
# Tips for publication-quality figures:
print('Publication Tips:')
print('1. Use vector formats: .pdf, .svg, .eps')
print('2. DPI ≥ 300 for raster images')
print('3. Font sizes: axis labels 11-12pt, titles 14pt')
print('4. Line widths: 1.5-2.0 for main plots')
print('5. Color-blind friendly palettes')
print('6. Remove chart junk (unnecessary gridlines, etc.)')
print('\nExample exports:')
print('  plt.savefig("figure.pdf", dpi=300, bbox_inches="tight")')
print('  plt.savefig("figure.svg", bbox_inches="tight")')

## Summary

### Best Practices
1. **Corner plots**: Show parameter correlations
2. **Uncertainty bands**: Always show confidence intervals
3. **Styling**: Consistent, professional appearance
4. **Formats**: PDF/SVG for papers, PNG for presentations
5. **Resolution**: 300+ DPI for print

### Tools Used
- `matplotlib`: Core plotting
- `corner`: Parameter correlations
- `src.viz_paper`: Project-specific visualizations

### Customization
- Color schemes: Match your institution/journal
- Fonts: Serif for papers, sans-serif for presentations
- Size: Adjust for single vs. double column

Congratulations! You've completed all tutorials. 🎉